# 請結合前面的知識與程式碼，比較不同的 optimizer 與 learning rate 組合對訓練的結果與影響
### 常見的 optimizer 包含
* SGD
* RMSprop
* AdaGrad
* Adam

In [8]:
import os
import keras
import itertools

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [9]:
train, test = keras.datasets.cifar10.load_data()

In [10]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [11]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [12]:
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    """
    Build your own model
    """
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [13]:
## 超參數設定
"""
Set your required experiment parameters
"""
LEARNING_RATE = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
EPOCHS = 50
BATCH_SIZE = 256
MOMENTUM = [0.7, 0.9, 0.95, 0.99]


In [ ]:
results = {}
"""
建立你的訓練與實驗迴圈並蒐集資料
"""
for lr, momentum in itertools.product(LEARNING_RATE, MOMENTUM):
    keras.backend.clear_session() # 把舊的 Graph 清掉
    print("Experiment with LR = %.6f, Momentum = %.2f" % (lr, momentum))
    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    optimizer = keras.optimizers.SGD(lr=lr, nesterov=True, momentum=momentum)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)
    
    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["accuracy"]
    valid_acc = model.history.history["val_accuracy"]
    
    exp_name_tag = "exp-lr-%s-momemtum-%s" % (str(lr), str(momentum))
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}

Experiment with LR = 0.100000, Momentum = 0.70
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
196/196 [==============================] - 3s 13ms/s

196/196 [==============================] - 2s 11ms/step - loss: 0.7658 - accuracy: 0.7205 - val_loss: 1.9547 - val_accuracy: 0.4814
Experiment with LR = 0.100000, Momentum = 0.90
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params:

196/196 [==============================] - 2s 12ms/step - loss: 1.1068 - accuracy: 0.6036 - val_loss: 1.6811 - val_accuracy: 0.4720
Epoch 50/50
196/196 [==============================] - 3s 13ms/step - loss: 1.1124 - accuracy: 0.6002 - val_loss: 1.7859 - val_accuracy: 0.4465
Experiment with LR = 0.100000, Momentum = 0.95
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_____________________________________________________________

196/196 [==============================] - 2s 11ms/step - loss: 1.5411 - accuracy: 0.4543 - val_loss: 1.6826 - val_accuracy: 0.4166
Epoch 49/50
196/196 [==============================] - 2s 11ms/step - loss: 1.5267 - accuracy: 0.4591 - val_loss: 1.6972 - val_accuracy: 0.4234
Epoch 50/50
196/196 [==============================] - 2s 11ms/step - loss: 1.5332 - accuracy: 0.4580 - val_loss: 1.6643 - val_accuracy: 0.4266
Experiment with LR = 0.100000, Momentum = 0.99
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________

196/196 [==============================] - 2s 10ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 50/50
196/196 [==============================] - 2s 10ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Experiment with LR = 0.010000, Momentum = 0.70
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output 

...

196/196 [==============================] - 2s 10ms/step - loss: 1.7082 - accuracy: 0.4062 - val_loss: 1.7092 - val_accuracy: 0.4074
Epoch 41/50
196/196 [==============================] - 2s 10ms/step - loss: 1.7040 - accuracy: 0.4076 - val_loss: 1.7057 - val_accuracy: 0.4071
Epoch 42/50
196/196 [==============================] - 2s 10ms/step - loss: 1.7002 - accuracy: 0.4079 - val_loss: 1.7023 - val_accuracy: 0.4078
Epoch 43/50
196/196 [==============================] - 2s 11ms/step - loss: 1.6967 - accuracy: 0.4093 - val_loss: 1.6979 - val_accuracy: 0.4122
Epoch 44/50
196/196 [==============================] - 2s 10ms/step - loss: 1.6930 - accuracy: 0.4118 - val_loss: 1.6950 - val_accuracy: 0.4140
Epoch 45/50
196/196 [==============================] - 2s 10ms/step - loss: 1.6893 - accuracy: 0.4126 - val_loss: 1.6912 - val_accuracy: 0.4147
Epoch 46/50
196/196 [==============================] - 2s 10ms/step - loss: 1.6857 - accuracy: 0.4137 - val_loss: 1.6878 - val_accuracy: 0.4147
Epoc

196/196 [==============================] - 2s 11ms/step - loss: 1.6057 - accuracy: 0.4403 - val_loss: 1.6128 - val_accuracy: 0.4379
Epoch 40/50
196/196 [==============================] - 2s 11ms/step - loss: 1.6006 - accuracy: 0.4438 - val_loss: 1.6087 - val_accuracy: 0.4411
Epoch 41/50
196/196 [==============================] - 2s 11ms/step - loss: 1.5952 - accuracy: 0.4453 - val_loss: 1.6033 - val_accuracy: 0.4376
Epoch 42/50
196/196 [==============================] - 2s 11ms/step - loss: 1.5899 - accuracy: 0.4468 - val_loss: 1.5994 - val_accuracy: 0.4384
Epoch 43/50
196/196 [==============================] - 2s 10ms/step - loss: 1.5857 - accuracy: 0.4484 - val_loss: 1.5961 - val_accuracy: 0.4431
Epoch 44/50
196/196 [==============================] - 2s 10ms/step - loss: 1.5810 - accuracy: 0.4512 - val_loss: 1.5910 - val_accuracy: 0.4464
Epoch 45/50
196/196 [==============================] - 2s 11ms/step - loss: 1.5760 - accuracy: 0.4505 - val_loss: 1.5867 - val_accuracy: 0.4462
Epoc

196/196 [==============================] - 2s 11ms/step - loss: 1.2450 - accuracy: 0.5636 - val_loss: 1.3688 - val_accuracy: 0.5052
Epoch 39/50
196/196 [==============================] - 2s 11ms/step - loss: 1.2369 - accuracy: 0.5636 - val_loss: 1.3640 - val_accuracy: 0.5108
Epoch 40/50
196/196 [==============================] - 2s 10ms/step - loss: 1.2274 - accuracy: 0.5663 - val_loss: 1.3609 - val_accuracy: 0.5144
Epoch 41/50
196/196 [==============================] - 2s 10ms/step - loss: 1.2197 - accuracy: 0.5715 - val_loss: 1.3698 - val_accuracy: 0.5047
Epoch 42/50
196/196 [==============================] - 2s 10ms/step - loss: 1.2122 - accuracy: 0.5738 - val_loss: 1.3532 - val_accuracy: 0.5177
Epoch 43/50
196/196 [==============================] - 2s 10ms/step - loss: 1.2002 - accuracy: 0.5793 - val_loss: 1.3515 - val_accuracy: 0.5213
Epoch 44/50
196/196 [==============================] - 2s 10ms/step - loss: 1.1913 - accuracy: 0.5807 - val_loss: 1.3485 - val_accuracy: 0.5213
Epoc

196/196 [==============================] - 2s 11ms/step - loss: 2.2049 - accuracy: 0.1993 - val_loss: 2.2061 - val_accuracy: 0.2032
Epoch 38/50
196/196 [==============================] - 2s 11ms/step - loss: 2.2028 - accuracy: 0.2014 - val_loss: 2.2041 - val_accuracy: 0.2043
Epoch 39/50
196/196 [==============================] - 2s 11ms/step - loss: 2.2007 - accuracy: 0.2032 - val_loss: 2.2020 - val_accuracy: 0.2067
Epoch 40/50
196/196 [==============================] - 2s 11ms/step - loss: 2.1987 - accuracy: 0.2046 - val_loss: 2.2000 - val_accuracy: 0.2077
Epoch 41/50
196/196 [==============================] - 2s 11ms/step - loss: 2.1967 - accuracy: 0.2065 - val_loss: 2.1980 - val_accuracy: 0.2098
Epoch 42/50
196/196 [==============================] - 2s 11ms/step - loss: 2.1947 - accuracy: 0.2081 - val_loss: 2.1960 - val_accuracy: 0.2116
Epoch 43/50
196/196 [==============================] - 2s 11ms/step - loss: 2.1928 - accuracy: 0.2100 - val_loss: 2.1940 - val_accuracy: 0.2121
Epoc

196/196 [==============================] - 2s 11ms/step - loss: 2.0673 - accuracy: 0.2813 - val_loss: 2.0640 - val_accuracy: 0.2857
Epoch 37/50
196/196 [==============================] - 2s 11ms/step - loss: 2.0633 - accuracy: 0.2820 - val_loss: 2.0602 - val_accuracy: 0.2872
Epoch 38/50
196/196 [==============================] - 2s 10ms/step - loss: 2.0595 - accuracy: 0.2842 - val_loss: 2.0564 - val_accuracy: 0.2870
Epoch 39/50
196/196 [==============================] - 2s 11ms/step - loss: 2.0557 - accuracy: 0.2853 - val_loss: 2.0527 - val_accuracy: 0.2882
Epoch 40/50
196/196 [==============================] - 2s 11ms/step - loss: 2.0520 - accuracy: 0.2866 - val_loss: 2.0491 - val_accuracy: 0.2899
Epoch 41/50
196/196 [==============================] - 2s 10ms/step - loss: 2.0484 - accuracy: 0.2877 - val_loss: 2.0455 - val_accuracy: 0.2921
Epoch 42/50
196/196 [==============================] - 2s 11ms/step - loss: 2.0449 - accuracy: 0.2897 - val_loss: 2.0420 - val_accuracy: 0.2926
Epoc

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

"""
將實驗結果繪出
"""
NUM_COLORS = len(results.keys())
cm = plt.get_cmap('gist_rainbow')
color_bar = [cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()